# Road type data to area of prediction

This notebooks integrates proximity to different road types to the area of prediction

## Import libraries

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

import distancerasters as dr

import matplotlib.pyplot as plt
import seaborn as sns

import aup

## Data download/read and preprocessing

### Area of prediction with variables

In [2]:
aop_gdf = gpd.read_file('../../data/processed/prediccion_uso_suelo/tess_kde.geojson')
print(aop_gdf.shape)
aop_gdf.head(2)

(294835, 28)


,fid,uso_suelo,area_m2,full_plus_code,CVEGEO,bld_area_m2,block_area_m2,pred_area_pct,pobtot,cultural_recreativo,...,uso_tot,pct_habitacional,pct_cultural_recreativo,pct_servicios,pct_comercio,pct_salud,pct_educacion,pct_gobierno,pct_industria,geometry
0,0,Servicio,2747.599834,75GRMHR2+458V,1412000011206018,173.7348,278416.454431,0.009869,0,0.0,...,4.926511,0.0,0.0,0.740466,0.231342,0.0,0.0,0.0,0.028192,"POLYGON ((2349620.239 965525.905, 2349619.865 ..."
1,1,Sin dato,900.765482,75GRMGRX+GQP3,1412000011206018,81.7922,278416.454431,0.003235,0,0.0,...,58.144456,0.0,0.0,0.586683,0.354285,0.0,0.0,0.0,0.059033,"POLYGON ((2349506.214 965639.352, 2349505.838 ..."


### Area of analysis

In [3]:
aoa_gdf = gpd.read_file("../../data/processed/prediccion_uso_suelo/Distrito/distrito.shp")
aoa_gdf = aoa_gdf.to_crs("EPSG:4326")
print(aoa_gdf.shape)
aoa_gdf.head(2)

(1, 2)


,id,geometry
0,1,"POLYGON ((-103.4233 20.91026, -103.42209 20.91..."


### Edges

In [4]:
schema = "osmnx"
table = "edges_osmnx_23_line"

edges = aup.gdf_from_polygon(aoa_gdf, schema, table)
print(edges.shape)
edges.head(2)

(146103, 19)


,osmid,v,u,key,oneway,lanes,name,highway,maxspeed,length,geometry,bridge,ref,junction,tunnel,access,width,service,city
0,701379894,1742199439,1665230207,0,False,None,Privada Fresno,residential,None,85.256,"LINESTRING (-103.42168 20.58715, -103.42172 20...",None,None,None,None,None,None,None,Guadalajara
1,467372703,1873910987,1665442630,0,False,None,Avenida Los Cajetes,residential,None,68.582,"LINESTRING (-103.42383 20.58681, -103.42389 20...",None,None,None,None,None,None,None,Guadalajara


#### Road type processing

Transform lists stored as strings to list

In [5]:
from ast import literal_eval

In [6]:
def check_for_lists(val):
    try:
        val = literal_eval(val)
        return val
    except:
        return val

In [7]:
edges['highway'] = edges.highway.apply(lambda row: check_for_lists(row))

In [8]:
edges.loc[edges.highway.map(lambda x: 
                            isinstance(x, list)),'highway'] = edges.loc[
    edges.highway.map(lambda x: isinstance(x, list))].apply(
        lambda row: row['highway'][0], axis=1)

Check process

In [9]:
len(edges.loc[edges.highway.map(lambda x: 
                            isinstance(x, list))])

0

#### Remove link sections

In [10]:
edges.loc[edges['highway'].str.contains(
    "_link"),'highway'] = edges[edges['highway'].str.contains(
    "_link")].highway.apply(lambda x: x.replace('_link',''))

Check result

In [11]:
len(edges.loc[edges['highway'].str.contains(
    "_link"),'highway'])

0

## Road type proximity to area of analysis

#### Test the algorithm

Rasterize road data

In [12]:
edges = edges.to_crs("EPSG:4326")

In [13]:
edges_test = edges.loc[edges.highway=='primary'].copy()
print(edges_test.shape)
edges_test.head(2)

(2455, 19)


,osmid,v,u,key,oneway,lanes,name,highway,maxspeed,length,geometry,bridge,ref,junction,tunnel,access,width,service,city
54,653450225,6148237946,4620035014,0,True,3,Antiguo Camino Real a Colima,primary,None,16.516,"LINESTRING (-103.42038 20.58717, -103.4205 20....",None,None,None,None,None,None,None,Guadalajara
65,653450225,8694189056,6148237946,0,True,3,Antiguo Camino Real a Colima,primary,None,241.384,"LINESTRING (-103.4205 20.58707, -103.42231 20....",None,None,None,None,None,None,None,Guadalajara


In [14]:
pixel_size = 0.00023 # 0.00023° -> 25m

In [15]:
bounds = []
for c in edges_test.bounds:
    if 'min' in c:
        bounds.append(edges_test.bounds[c].min().item())
    else:
        bounds.append(edges_test.bounds[c].max().item())
bounds = tuple(bounds)
bounds

(-103.5399068, 20.5841918, -103.3371528, 20.9004883)

In [16]:
output_dir = '../../data/processed/prediccion_uso_suelo/prox_vialidades/'
rv_array, affine = dr.rasterize(edges_test, pixel_size=pixel_size,
                                bounds=bounds, 
                                output=output_dir+"primary_rasterized_binary.tif")

In [17]:
affine

Affine(0.00023, 0.0, -103.54002,
       0.0, -0.00023, 20.90056)

Proximity raster

In [18]:
def raster_conditional(rarray):
    return (rarray == 1)

# generate distance array and output to geotiff
my_dr = dr.DistanceRaster(rv_array, affine=affine,
                          output_path=output_dir+"linestrings_distance_raster.tif",
                          conditional=raster_conditional)

Tree build time: 0.003 seconds
Distance calc run time: 12.075 seconds


In [19]:
my_dr

In [21]:
import rasterio

In [23]:
with rasterio.open(output_dir+"linestrings_distance_raster.tif") as src:
    affine = src.transform
    distance_raster = src.read(1)

In [24]:
distance_raster

array([[10892.08162625, 10869.12487121, 10846.1721656 , ...,
         9868.65606056,  9892.54534059,  9916.4346345 ],
       [10885.03114575, 10862.05945444, 10839.09175198, ...,
         9868.29908858,  9892.18926726,  9916.07945559],
       [10878.03622343, 10855.04969406, 10832.06709326, ...,
         9868.00838593,  9891.89930328,  9915.79022679],
       ...,
       [11181.69454695, 11160.32909789, 11138.97411331, ...,
         8273.07995895,  8292.62491198,  8312.19284929],
       [11193.24804307, 11171.90465701, 11150.57180914, ...,
         8287.8946538 ,  8307.40478155,  8326.93801632],
       [11204.84800027, 11183.52672017, 11162.21605183, ...,
         8302.76169244,  8322.23699625,  8341.73552896]],
      shape=(1375, 883))